In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")


In [21]:
# Load dataset
data = pd.read_csv("100_Sales.csv")  

# Encode categorical variables if any
label_encoders = {}
for column in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


In [23]:
X = data.drop("Total_Profit", axis=1) 
y = data["Total_Profit"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
print(data.head())

   Region  Country  Item_Type  Sales_Channel  Order_Priority  Ship_Date  \
0       1       73          0              0               1         81   
1       2       22          2              1               0         43   
2       3       55          8              0               2          8   
3       6       59          5              1               0         18   
4       6       56          8              0               2          2   

   Unit_Cost  Total_Revenue  Total_Profit  Unnamed: 9  Unnamed: 10  
0     159.42     2533654.00     951410.50         NaN          NaN  
1     117.11      576782.80     248406.36         NaN          NaN  
2     524.96     1158502.59     224598.75         NaN          NaN  
3       6.92       75591.66      19525.82         NaN          NaN  
4     524.96     3296425.02     639077.50         NaN          NaN  


In [27]:
# Binning total_profit into classes

# Define bins and labels for categorizing
bins = [-float('inf'), 0, 1000, float('inf')]  
labels = ['Low Profit', 'Medium Profit', 'High Profit']  

# Apply binning to create categorical target
data['profit_category'] = pd.cut(data['Total_Profit'], bins=bins, labels=labels)

# Update the target variable
X = data.drop(['Total_Profit', 'profit_category'], axis=1)
y = data['profit_category']


In [29]:

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:

# Initialize and train the Decision Tree classifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = dt_model.predict(X_test)

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))


Decision Tree Accuracy: 1.0
Decision Tree Classification Report:
               precision    recall  f1-score   support

 High Profit       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Decision Tree Confusion Matrix:
 [[20]]


In [33]:

# Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


Random Forest Accuracy: 1.0
Random Forest Classification Report:
               precision    recall  f1-score   support

 High Profit       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Random Forest Confusion Matrix:
 [[20]]


In [35]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get best parameters and model
best_rf_model = grid_search.best_estimator_

# Evaluate the tuned model
y_pred_best_rf = best_rf_model.predict(X_test)
print("Tuned Random Forest Accuracy:", accuracy_score(y_test, y_pred_best_rf))
print("Tuned Random Forest Classification Report:\n", classification_report(y_test, y_pred_best_rf))


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Tuned Random Forest Accuracy: 1.0
Tuned Random Forest Classification Report:
               precision    recall  f1-score   support

 High Profit       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

